# Guidelines for Prompting
In this lesson, you'll practice two prompting principles and their related tactics in order to write effective prompts for large language models.

## Setup
#### Load the API key and relevant Python libaries if needed.

In [1]:
%pip install -qU openai


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip show openai

Name: openai
Version: 0.27.8
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: /home/limcheekin/ws/py/prompt-engineering-for-developers/venv/lib/python3.10/site-packages
Requires: aiohttp, requests, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
import openai
import os

openai.api_key  = os.getenv('OPENAI_API_KEY')

__OpenAI API helper function__

Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). (llama-cpp-python with Wizard-Vicuna-13B-Uncensored-GGML model at the back)

This helper function will make it easier to use prompts and look at the generated outputs:

In [4]:
# Orca Mini Prompt
DEFAULT_SYSTEM_PROMPT = "You are a helpful assistant, you will complete the task by follow the instructions given."
def get_prompt(system=DEFAULT_SYSTEM_PROMPT, instruction=None, input=None):
    if not instruction:
        raise ValueError("instruction must be specified")
    if input:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    else:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Response:\n"
    
    return prompt

# REF: https://platform.openai.com/docs/api-reference/chat/create
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": get_prompt(instruction=prompt)}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=1000,
    )
    return response.choices[0].message["content"]    

In [5]:
get_completion("What is AI?")

'\n AI stands for Artificial Intelligence, which is a branch of computer science that focuses on creating intelligent machines that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI systems use algorithms and statistical models to analyze large amounts of data and learn from it, allowing them to improve their performance over time and adapt to new situations. Some popular applications of AI include virtual assistants like Siri and Alexa, image and speech recognition in smartphones and security systems, and self-driving cars.'

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [6]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence:.
```{text}```
"""
response = get_completion(prompt)
print(response)


You should express what you want a model to do by providing instructions that are as clear and specific as you can possibly make them. This will guide the model towards the desired output, reduce the chances of receiving irrelevant or incorrect responses. Don't confuse writing a clear prompt with writing a short prompt. In many cases, longer prompts provide more clarity and context for the model, which can lead to more detailed and relevant outputs.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [7]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)


Here are three made-up book titles along with their authors and genres in JSON format:

```json
{
 "book_id": "1",
 "title": "The Last One Standing",
 "author": "John Smith",
 "genre": "Fiction"
},
{
 "book_id": "2",
 "title": "The Secret Treasure Hunt",
 "author": "Jane Doe",
 "genre": "Mystery"
},
{
 "book_id": "3",
 "title": "The Time Traveler's Journey",
 "author": "Bob Johnson",
 "genre": "Science Fiction"
}
```


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [8]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:


Step 1 - Get some water boiling.
Step 2 - Put a tea bag in a cup.
Step 3 - Pour the hot water over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - Take out the tea bag.
Step 6 - If you like, add some sugar or milk to taste.
Step N - Enjoy your delicious cup of tea!


In [9]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:


Step 1 - Enjoy the beautiful weather by going for a walk in the park.
Step 2 - Take in the sights and sounds of nature by listening to the birds singing and watching the flowers bloom.
Step N - Spend time outdoors and appreciate the beauty of nature.


#### Tactic 4: "Few-shot" prompting

In [10]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

 Great! Here's my consistent style answer to your task:

Teach me about patience. Patience means being able to wait for something without getting upset or impatient. It takes practice to develop patience, but it is an important skill to have in life. When we have patience, we can handle challenges and setbacks in a more calm and rational way. We can also be more understanding and patient with others, especially when they are going through difficult times. Developing patience can help us lead happier and more fulfilling lives.


### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [11]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:

1 - Summary in French: "Jack et Jill ont embarqué sur une quête pour aller chercher de l'eau du sommet d'un mont. Ils ont emprunté la route joyeusement, alors que le malheur s'est abattu : Jack a perdu de l'énergie en se fissant une souris et Jill suit suite. Malgré cette mauvaise chance, les esprits d'aventurenels ont demeuré intenses et ils se sont continuement emparcés."
2 - Noms dans la summary française: "Jack et Jill"
3 - Liste des noms dans la summary française: "Jack et Jill"
4 - Objet de données JSON qui contient les étiquettes suivantes : "franchise_summary", "num_names".


#### Ask for output in a specified format

In [12]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:

Summary: Jack and Jill went on a quest to fetch water from a hilltop well. They sang joyfully while climbing but Jack tripped on a stone and tumbled down the hill, followed by Jill. Although slightly battered, they continued exploring with delight.
Translation: Une charmante ville, les frères et les sœurs Jack et Jill ont emprunté une quête pour aller chercher de l'eau de la source d'un mont. Ils ont chanté joyeusement pendant leur ascension, mais Jack a perdu son balancement lorsqu'il a heurté un rocher et est tombé en bas de la colline, suivi de Jill. Malheureusement, ils sont sortis de l'accident avec quelques soucis de forme, mais leur esprit d'aventure reste intact.
Noms: Jack, Jill

Output JSON: {"french_summary": "Jack et Jill ont emprunté une quête pour aller chercher de l'eau de la source d'un mont.", "num_names": ["Jack", "Jill"]}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [13]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)


To determine if the student's solution is correct or not, we need to calculate the total cost for the first year of operations as a function of the number of square feet. 

The student's solution suggests that the total cost for the first year of operations as a function of the size of the installation (x) is:

450x + 100,000

To verify this formula, we can plug in some values for x and check if the resulting total cost matches the expected value. 

Let's start with an example where x = 20 square feet. 

Plugging in these values, we get:

450(20) + 100,000 = 7,500 + 100,000 = 107,500

This matches the expected total cost of $100,000 for the first year of operations as a function of x = 20 square feet. 

We can repeat this process for different values of x and compare the resulting total costs to the expected values. If all the calculations match, we can conclude that the student's solution is correct.


#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.

In [17]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)



Question:
I'm building a solar power installation and I need help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.

Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```

Actual solution:

Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```

Is the student's solution the same as actual solution just calculated:
Yes.


## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

In [15]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

 

AeroGlide UltraSlim Smart Toothbrush by Boie is a high-end toothbrush that is designed to provide an exceptional dental hygiene experience. It features a unique design that allows for easy and comfortable brushing, with a slim and sleek handle that fits comfortably in your hand. The toothbrush has been ergonomically designed to provide a perfect fit for your fingers, making it easy to hold and use.

The AeroGlide UltraSlim Smart Toothbrush by Boie is equipped with a powerful motor that provides efficient and effective cleaning. It also comes with a variety of innovative features, such as pressure sensors that detect the right brushing pressure for your teeth, and an intelligent timer that ensures you brush for the recommended two minutes.

The toothbrush is made from high-quality materials that are durable and long-lasting. It also has a compact and lightweight design, making it easy to carry around and use on-the-go.

Overall, the AeroGlide UltraSlim Smart Toothbrush by Boie is an 

## Try experimenting on your own!

#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.

# Iterative Prompt Development
In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

## Generate a marketing product description from a product fact sheet

In [18]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [19]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)

response

'\n\nProduct Description:\n\nIntroducing the SWC-100 Mid-Century inspired office chair, a beautiful addition to any home or business setting. This chair is part of a family of office furniture, including filing cabinets, desks, bookcases, meeting tables, and more. The chair is available with or without armrests and can be customized with several options of shell color and base finishes.\n\nThe SWC-100 chair features a 5-wheel plastic coated aluminum base with pneumatic chair adjust for easy raise/lower action. The chair is suitable for contract use and qualified for home or business settings. It is made with cast aluminum with modified nylon PA6/PA66 coating and has a shell thickness of 10 mm.\n\nThe SWC-100 chair comes with two choices of seat foam densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3). It also has armless or 8 position PU armrests options. The chair is designed in Italy and is a perfect addition to any home or business setting.\n\nIn summary, the SWC-100 Mid-Century ins

## Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [20]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)

response

"\n\nProduct Description:\nThe SWC-100 office chair is a beautiful addition to any home or business setting. With its mid-century inspired design, it's sure to add a touch of style to your workspace. The chair features a 5-wheel plastic coated aluminum base with pneumatic adjustability for easy raise/lower action. The chair is suitable for contract use and qualified for home or business settings.\n\nThe chair has several options of shell color and base finishes, including stainless steel, matte black, gloss white, or chrome. It's available with or without armrests and comes with two choices of seat foam densities: medium or high. The chair is made with a cast aluminum base with modified nylon PA6/PA66 coating and has a shell thickness of 10 mm.\n\nThe SWC-100 chair is perfect for any home or business setting and can be customized to fit your needs. Its soft or hard-floor caster options, two choices of seat foam densities, and armless or 8 position PU armrests make it a versatile additi

In [21]:
len(response.split(" "))

165

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [22]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)

response

'\n\nProduct Description:\n\nThe SWC-100 office chair is a beautiful addition to any mid-century inspired home or business setting. With several options of shell color and base finishes, this chair is perfect for any environment. The base finish options are stainless steel, matte black, gloss white, or chrome, ensuring that it will complement any space. The chair is also available with or without armrests, making it even more versatile. \n\nThe construction of the SWC-100 chair is made with a 5-wheel plastic coated aluminum base and pneumatic chair adjust for easy raise/lower action. The shell is made with a cast aluminum frame with a modified nylon PA6/PA66 coating, providing a comfortable seat with a thickness of 10 mm. The seat is made with HD36 foam, ensuring maximum comfort for the user.\n\nThe SWC-100 chair is suitable for home or business settings and is qualified for contract use. It comes with soft or hard-floor caster options and two choices of seat foam densities: medium (1.

In [23]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)

response

'\n\nProduct Description:\n\nThe SWC-100 office chair is a beautiful addition to any mid-century inspired home or business setting. With several options of shell color and base finishes, this chair is perfect for any environment. The pneumatic chair adjust feature makes it easy to raise and lower the seat height, while the 5-wheel plastic coated aluminum base ensures stability.\n\nThe chair is constructed with a cast aluminum base with a modified nylon PA6/PA66 coating, providing a comfortable seat with a hard or soft floor caster option. The shell thickness is 10 mm, and the seat foam density can be adjusted to medium or high depending on preference. The armless design is available with or without armrests, and the Product ID is SWC-100.\n\nThis chair is suitable for home or business settings and is qualified for contract use. The materials used in this product are cast aluminum with a modified nylon PA6/PA66 coating and HD36 foam. The country of origin for this product is Italy.'

## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [24]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)

response

'\n\nProduct Description:\n\nIntroducing the mid-century inspired office furniture, the SWC-100 filing cabinet. This product is part of a beautiful family of office furniture, including desks, bookcases, meeting tables, and more. The SWC-100 filing cabinet is available in several options of shell color and base finishes. The base finish options are stainless steel, matte black, gloss white, or chrome. The chair is available with or without armrests and is suitable for home or business settings. This product is also qualified for contract use.\n\nProduct Dimensions:\n\nThe SWC-100 filing cabinet has the following dimensions:\n\n- WIDTH 53 CM | 20.87”\n- DEPTH 51 CM | 20.08”\n- HEIGHT 80 CM | 31.50”\n- SEAT HEIGHT 44 CM | 17.32”\n- SEAT DEPTH 41 CM | 16.14”\n\nTable of Dimensions:\n\n| Name of Dimension | Measurements in Inches |\n|-----------------------|--------------------------|\n| Height | 80 cm |\n| Width | 53 cm |\n| Depth | 51 cm |\n| Seat Height | 44 cm |\n| Seat Depth | 41 cm |

## Load Python libraries to view HTML

In [25]:
from IPython.display import display, HTML
display(HTML(response))

## Try experimenting on your own!

# Summarizing
In this lesson, you will summarize text with a focus on specific topics.

## Text to summarize

In [27]:
prod_review = """
Got this panda plush toy for my daughter's birthday, \
who loves it and takes it everywhere. It's soft and \ 
super cute, and its face has a friendly look. It's \ 
a bit small for what I paid though. I think there \ 
might be other options that are bigger for the \ 
same price. It arrived a day earlier than expected, \ 
so I got to play with it myself before I gave it \ 
to her.
"""

## Summarize with a word/sentence/character limit

In [28]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)



Review: This panda plush toy is loved by the reviewer's daughter and is soft and super cute with a friendly face. However, the size is a bit small for the price paid, and there might be bigger options available at the same price. The reviewer received it a day earlier than expected and got to play with it before giving it to their daughter.


## Summarize with a focus on shipping and delivery

In [29]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
Shipping deparmtment. 

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that mention shipping and delivery of the product. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)




Got this panda plush toy for my daughter's birthday, who loves it and takes it everywhere. It's soft and super cute, and its face has a friendly look. It's a bit small for what I paid though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to her.


## Summarize with a focus on price and value

In [30]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site to give feedback to the \
pricing deparmtment, responsible for determining the \
price of the product.  

Summarize the review below, delimited by triple 
backticks, in at most 30 words, and focusing on any aspects \
that are relevant to the price and perceived value. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)




Summary: This customer bought a soft and cute panda plush toy for their daughter's birthday. Although the toy is small, the customer's daughter loves it and takes it everywhere. The toy has a friendly face and arrived a day earlier than expected, allowing the customer to play with it before giving it to their daughter.


#### Comment
- Summaries include topics that are not related to the topic of focus.

## Try "extract" instead of "summarize"

In [31]:
prompt = f"""
Your task is to extract relevant information from \ 
a product review from an ecommerce site to give \
feedback to the Shipping department. 

From the review below, delimited by triple quotes \
extract the information relevant to shipping and \ 
delivery. Limit to 30 words. 

Review: ```{prod_review}```
"""

response = get_completion(prompt)
print(response)



Extracted information relevant to shipping and delivery:
- Product name: Panda plush toy
- Size: Small
- Material: Soft and cute
- Shipping date: Arrived a day earlier than expected
- Delivered to: Daughter or recipient


## Summarize multiple product reviews

In [32]:

review_1 = prod_review 

# review for a standing lamp
review_2 = """
Needed a nice lamp for my bedroom, and this one \
had additional storage and not too high of a price \
point. Got it fast - arrived in 2 days. The string \
to the lamp broke during the transit and the company \
happily sent over a new one. Came within a few days \
as well. It was easy to put together. Then I had a \
missing part, so I contacted their support and they \
very quickly got me the missing piece! Seems to me \
to be a great company that cares about their customers \
and products. 
"""

# review for an electric toothbrush
review_3 = """
My dental hygienist recommended an electric toothbrush, \
which is why I got this. The battery life seems to be \
pretty impressive so far. After initial charging and \
leaving the charger plugged in for the first week to \
condition the battery, I've unplugged the charger and \
been using it for twice daily brushing for the last \
3 weeks all on the same charge. But the toothbrush head \
is too small. I’ve seen baby toothbrushes bigger than \
this one. I wish the head was bigger with different \
length bristles to get between teeth better because \
this one doesn’t.  Overall if you can get this one \
around the $50 mark, it's a good deal. The manufactuer's \
replacements heads are pretty expensive, but you can \
get generic ones that're more reasonably priced. This \
toothbrush makes me feel like I've been to the dentist \
every day. My teeth feel sparkly clean! 
"""

# review for a blender
review_4 = """
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

reviews = [review_1, review_2, review_3, review_4]

In [33]:
for i in range(len(reviews)):
    prompt = f"""
    Your task is to generate a short summary of a product \ 
    review from an ecommerce site. 

    Summarize the review below, delimited by triple \
    backticks in at most 20 words. 

    Review: ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")


0 

Summary: This panda plush toy is loved by the reviewer's daughter and is soft and cute with a friendly face. However, the reviewer paid a bit less for a smaller size, but there might be bigger options for the same price. The toy arrived a day earlier than expected, allowing the reviewer to play with it before giving it to their daughter. 

1 

Summary: This person needed a lamp for their bedroom with storage and a reasonable price point. They received it quickly and the string broke during transit, but the company replaced it with a new one and sent the missing part quickly. Overall, they found the company to be helpful and caring about their customers and products. 

2  

Summary: The reviewer has been using an electric toothbrush for three weeks and likes it because it has a long battery life. However, the toothbrush head is too small and the manufacturer's replacements are expensive. Overall, this toothbrush is a good deal if you can get it around $50. 

3  

Here is a summary o

## Try experimenting on your own!

# Inferring
In this lesson, you will infer sentiment and topics from product reviews and news articles.

## Product review text

In [34]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

## Sentiment (positive/negative)

In [35]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

 The sentiment of the product review is positive.


In [36]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

 The sentiment of the product review is "positive".


## Identify types of emotions

In [37]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


Emotions expressed by the writer in the review are:
- Satisfaction
- Happiness
- Confidence
- Trustworthiness
- Responsiveness


## Identify anger

In [38]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

 Yes, the writer of the review is expressing anger.


## Extract product and company name from customer reviews

In [39]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


{
 "Item": "Lamp",
 "Brand": "Lumina"
}


## Doing multiple tasks at once

In [40]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


{
 "Sentiment": "Positive",
 "Anger": true,
 "Item": "Lamp",
 "Brand": "Lumina"
}


## Inferring topics

In [41]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

## Infer 5 topics

In [42]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)



The five topics being discussed in the text are:

1. Satisfaction rating of NASA employees
2. Popularity of NASA as a department
3. John Smith's comment on the survey findings
4. Director Tom Johnson's statement on the survey results
5. The government's response to the survey and its commitment to improving job satisfaction across all departments.


In [43]:
response.split(sep=',')

["\n\nThe five topics being discussed in the text are:\n\n1. Satisfaction rating of NASA employees\n2. Popularity of NASA as a department\n3. John Smith's comment on the survey findings\n4. Director Tom Johnson's statement on the survey results\n5. The government's response to the survey and its commitment to improving job satisfaction across all departments."]

In [44]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

## Make a news alert for certain topics

In [45]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: '''{story}'''
"""
response = get_completion(prompt)
print(response)



The list of topics in the given text are:
- NASA
- Local government
- Engineering
- Employee satisfaction
- Federal government


In [46]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

IndexError: list index out of range

## Try experimenting on your own!

# Transforming

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.


## Translation

LLM is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.


In [47]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)




```Hola, quiero pedir una licuadora. ¿Cómo se dice esto en español?```


In [48]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)


 The language of the given text is French.


In [49]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)




Translation for the given text in French:
```J'ai envie d'acheter de la balle pour jouer de basket.```

Translation for the given text in Spanish:
```Quiero pedir una bola de basketball para jugar al baloncesto.```

Translation for the given text in English:
```I want to order a basketball.```


In [50]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'
"""
response = get_completion(prompt)
print(response)


### Translate the following text to Spanish in both the formal and informal forms: 
'Would you like to order a pillow?'

Formal Form: ¿Te gustaría ordenar una silla?
Informal Form: ¿Te gusta ordenar una silla?


### Universal Translator

Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!


In [51]:
user_messages = [
    # System performance is slower than normal
    "La performance du système est plus lente que d'habitude.",
    # My monitor has pixels that are not lighting
    "Mi monitor tiene píxeles que no se iluminan.",
    # My mouse is not working
    "Il mio mouse non funziona",
    # My keyboard has a broken control key
    "Mój klawisz Ctrl jest zepsuty",
    "我的屏幕在闪烁"                                               # My screen is flashing
]


In [52]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")


Original message ( The language of the text is French.): La performance du système est plus lente que d'habitude.


Translation of the text in English: The system's performance is slower than usual.

Translation of the text in Korean: 시험은 더 긴 후에 종입되었습니다. (Siheon eun dae gil geun hyeok esujeotda)

Translation of the text in English: The experiment was completed faster than usual.

Translation of the text in Korean: 현기에 대해 일하는 것은 종입된 것이다. (Hyeongi eun dae ham neotdaen) 

Original message ( The language of the text is Spanish.): Mi monitor tiene píxeles que no se iluminan.


Translation of the text in English: "My monitor has pixels that are not illuminated."

Translation of the text in Korean: 

[ 내 센터에서 있는 플레이트 가요움 ](https://en.wikipedia.org/wiki/Monitor) 

[ 내 센터에서 있는 플레이트 가요움 ](https://en.wikipedia.org/wiki/Monitor) 

Original message ( The language of the text "Il mio mouse non funziona" appears to be Italian.): Il mio mouse non funziona


Translation of the text in English: "My mous

## Try it yourself!

Try some translations on your own!


## Tone Transformation

Writing can vary based on the intended audience. ChatGPT can produce different tones.


In [53]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)




Dear [Recipient],

I am writing to inform you about an interesting piece of furniture that I have come across. As a matter of fact, I was just discussing this standing lamp with my colleague, Joe, and he believes it would be perfect for our upcoming project.

The lamp in question is a Spec brand, model number [insert model number], and it comes with a one-year warranty. It is made of high-quality materials and has a sleek design that would complement any room decor.

I would love to hear your thoughts on this standing lamp and if you think it would be a good addition to our office. Please let me know if you have any questions or concerns.

Thank you for your time and consideration.

Best regards,
[Your Name]


## Format Conversion

ChatGPT can translate between formats. The prompt should describe the input and output formats.


In [54]:
data_json = {"resturant employees": [
    {"name": "Shyam", "email": "shyamjaiswal@gmail.com"},
    {"name": "Bob", "email": "bob32@gmail.com"},
    {"name": "Jai", "email": "jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)




To translate the given Python dictionary from JSON to an HTML table with column headers and title, you can follow these steps:

1. Create a Python dictionary with the values from the JSON data.
2. Use the `json` module in Python to convert the dictionary to JSON format.
3. Store the JSON data in a variable.
4. Use the `htmltable` module in Python to create an HTML table with the column headers and title.
5. Render the HTML table using the `print` function.

Here's the code:

```python
import json
from htmltable import HTMLTable

# Create a Python dictionary with values from JSON data
data = {'resturant employees': [{'name': 'Shyam', 'email': 'shyamjaiswal@gmail.com'}, {'name': 'Bob', 'email': 'bob32@gmail.com'}, {'name': 'Jai', 'email': 'jai87@gmail.com'}]}

# Convert the dictionary to JSON format
json_data = json.dumps(data)

# Create an HTML table with column headers and title
table = HTMLTable()
table.add_header('Restaurant Employees', 'Employee Name', 'Email')

# Render the HTML 

In [55]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))


Restaurant Employees,Employee Name,Email
Shyam,shyamjaiswal@gmail.com,
Bob,bob32@gmail.com,
Jai,jai87@gmail.com,


## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response.

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.


In [56]:
text = [
    # The girl has a ball.
    "The girl with the black and white puppies have a ball.",
    "Yolanda has her notebook.",  # ok
    "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
    "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
    "Your going to need you’re notebook.",  # Homonyms
    "That medicine effects my ability to sleep. Have you heard of the butterfly affect?",  # Homonyms
    "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)



 Here is the corrected version of the text:

"The girl with the black and white puppies has a ball."

I corrected the spelling errors by replacing "black" with "black" and "white" with "white". Additionally, I removed the extra punctuation around the text.


The corrected version of the text is:

"Yolanda has her notebook."

Here is the corrected version of the text:

"It's going to be a long day. Does the car need its oil changed?" 

No errors found.

Here is the corrected version of the text:

"There going to bring their suitcases."

I corrected the spelling errors by replacing "their" with "there" and "going" with "going". I also added a period at the end of the sentence to make it a complete thought.

Here is the corrected version of the text:

"And rewrite the corrected version. If you don't find any errors, just say 'No errors found'. Don't use any punctuation around the text."

 Here is the corrected version:

"That medicine affects my ability to sleep. Have you heard of the bu

In [57]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)




Here's the corrected review:

Got this for my daughter for her birthday cuz she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [58]:
%pip install redlines


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [59]:
from redlines import Redlines

diff = Redlines(text, response)
display(Markdown(diff.output_markdown))


<span style="color:red;font-weight:700;">Here's the corrected review: </span>

<span style="color:red;font-weight:700;"></span>Got this for my daughter for her birthday cuz she keeps taking mine from my <span style="color:red;font-weight:700;text-decoration:line-through;">room.  </span><span style="color:red;font-weight:700;">room. </span>Yes, adults also like pandas <span style="color:red;font-weight:700;text-decoration:line-through;">too.  </span><span style="color:red;font-weight:700;">too. </span>She takes it everywhere with her, and it's super soft and <span style="color:red;font-weight:700;text-decoration:line-through;">cute.  </span><span style="color:red;font-weight:700;">cute. </span>One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same <span style="color:red;font-weight:700;text-decoration:line-through;">price.  </span><span style="color:red;font-weight:700;">price. </span>It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

In [60]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))



Here is the corrected review:

Got this for my daughter for her birthday cuz she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

I hope this review meets your expectations and is written in APA style.

## Try it yourself!
Try changing the instructions to form your own review.

Thanks to the following sites:

https://writingprompts.com/bad-grammar-examples/


# Expanding
In this lesson, you will generate customer service emails that are tailored to each customer's review.

## Customize the automated reply to a customer email

In [62]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [63]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)



Dear Valued Customer,

Thank you for taking the time to share your recent experience with our product. We are sorry to hear that you were not satisfied with the price increase of our 17-piece blender system during the month of November and December.

We understand your frustration and apologize for any inconvenience caused. As a valued customer, we want to assure you that we take your feedback seriously and will investigate this matter further.

Please reach out to our customer service team if you have any further concerns or questions. We are committed to providing the best possible customer experience and will do everything in our power to resolve this issue.

Thank you for choosing our product and we hope to hear from you soon.

Best regards,

[Your Name]


## Use Temperature 0.1

In [65]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0.0):
    messages = [{"role": "user", "content": get_prompt(instruction=prompt)}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=1000,
    )
    return response.choices[0].message["content"]    

In [66]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt, temperature=0.1)
print(response)



Dear Valued Customer,

Thank you for taking the time to share your recent experience with our product. We are sorry to hear that you were not satisfied with the price increase of our 17-piece blender system and the 11-piece system.

We understand that this can be frustrating, especially if you plan to use these products frequently. We apologize for any inconvenience caused and assure you that we take customer satisfaction very seriously.

If you have any further concerns or questions, please do not hesitate to reach out to our customer service team. We are always here to assist you.

Thank you again for your feedback and we hope to hear from you soon.

Best regards,

[Your Name]


## Temparature

In [67]:
response = get_completion("my favorite food is", temperature=0.1)
print(response)

 I'm sorry, but as an AI language model, I don't have the ability to eat or have personal preferences. However, I can suggest some popular foods that people usually like such as pizza, burgers, fried chicken, sushi, pasta, and many more. Would you like me to provide you with some options?


In [69]:
response = get_completion("my favorite food is", temperature=1.0)
print(response)

 I'm sorry, but as an AI language model, I don't have the ability to eat or choose food preferences. However, if you want me to help with something else, please let me know.


In [70]:
response = get_completion("my favorite food is", temperature=2.0)
print(response)

 I'm sorry, but as an AI language model, I don't have personal preferences. But I can help you with various tasks related to writing, research, and providing information on various topics such as history, geography, science, etc.


## Try experimenting on your own!

# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

In [71]:

def format(messages=[], system="", user="Human: ", assistant="AI: "):
    """
    Format the messages from the API into human readable strings.
    """
    formatted_message = ""

    for message in messages:
        if message['role'] == 'system':
            formatted_message += f"{system}{message['content']}\n"
        elif message['role'] == 'user':
            formatted_message += f"{user}{message['content']}\n"
        elif message['role'] == 'assistant':
            formatted_message += f"{assistant}{message['content']}\n"

    return formatted_message


def get_completion_from_messages(messages=[], temperature=0.0):
    prompt = format(messages)
    return get_completion(prompt, temperature=temperature)

In [72]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'} ]

print(format(messages))

You are an assistant that speaks like Shakespeare.
Human: tell me a joke
AI: Why did the chicken cross the road
Human: I don't know



In [73]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

 Sure, here's a joke for you: "Why did the tomato turn red? Because it saw the salad dressing!"


In [74]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

 Hello, Isa! How can I assist you today?


In [75]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

 Sure, I am sorry to hear that you feel lost or confused. It's important to have a good understanding of who you are and where you stand in life. Could you please provide me with some details about yourself such as your full name, birthdate, place of birth, and any other relevant information? This will help me give you the most accurate and helpful assistance possible.


In [76]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

 
To add onto the conversation, could you please provide me with your full name so I can assist you better?


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant. 

In [77]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [78]:
import panel as pn  # GUI
pn.extension()


panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

/tmp/ipykernel_21729/3082588398.py:10: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.3, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


BokehModel(combine_events=True, render_bundle={'docs_json': {'335fb585-8b99-45ac-8d18-4934cdf9273f': {'version…

In [79]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=1)
print(response)

 Okay, let's start by creating a summary of the previous food order:

```
{
 "items": [
 {
 "name": "pizza",
 "size": "12",
 "toppings": ["pepperoni", "mushrooms", "onions"],
 "drinks": ["coke", "sprite", "bubbly water"],
 "price": 12.95
 },
 {
 "name": "pizza",
 "size": "10",
 "toppings": ["cheese", "eggplant"],
 "drinks": ["water", "fries", "greek salad"],
 "price": 9.75
 },
 {
 "name": "pizza",
 "size": "6",
 "toppings": ["mushrooms", "onions", "olives"],
 "drinks": ["sprite", "bubbly water", "water"],
 "price": 6.50
 },
 {
 "name": "fries",
 "size": "4",
 "price": 3.50
 }
 ]
}
```

As you can see, the previous order includes three types of pizza - pepperoni, cheese, and eggplant - as well as fries and a drink. The total price for all items is $32.40. 

Do you want to add anything else to your order?


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

## Overview
The notebook is created to tackle the problem of incomplete generation of sentence due to max_tokens settings.

Initial thought on solving the problem with the following ways:
1. Use stop sequences
2. Increase max_tokens
3. Use prompt to restrict the length/tokens of the generated text

### Use stop sequences

In [ ]:
response = get_completion("What is AI?", params={"stop": ["\n"]})
print(response)

### Increase max_tokens

In [ ]:
response = get_completion("What is AI?", params={"max_tokens": 512})
print(response)


### Use prompt to restrict the length/tokens of the generated text

In [ ]:
response = get_completion("What is AI? Limit the answer to 128 characters.", params={"max_tokens": 128})
print(response)

In [ ]:
response = get_completion("What is AI? Limit the answer to 128 tokens.", params={"max_tokens": 128})
print(response)

In [ ]:
response = get_completion("What is AI? Limit the answer to 60 words.", params={"max_tokens": 128})
print(response)

In [ ]:
response = get_completion("What is AI? Limit the answer to 60 words and make sure it is end in full sentence.", 
                          params={
                                "max_tokens": 128,
                                "stop": ["\n"]
                              })
print(response)

In [ ]:
response = get_completion("Limit the answer to 128 characters. What is AI?", params={"max_tokens": 128})
print(response)

In [ ]:
response = get_completion("Limit the answer to 128 tokens. What is AI?", params={"max_tokens": 128})
print(response)

In [ ]:
response = get_completion("Limit the answer to 60 words. What is AI?", params={"max_tokens": 128})
print(response)

In [ ]:
response = get_completion("Limit the answer to 60 words and make sure it is a complete sentence. What is AI?", params={"max_tokens": 128})
print(response)

In [ ]:
response = get_completion("Make sure the answer is end in full sentence. What is AI?", 
                          params={"max_tokens": 64})
print(response)

In [ ]:
response = get_completion("Answer the question. What is AI?", 
                          params={"max_tokens": 128})
print(response)

### Conclusion
Stop sequences is the preferred way to make sure the generated text most likely end in complete sentence. You can learn more about stop sequences at https://help.openai.com/en/articles/5072263-how-do-i-use-stop-sequences.

Otherwise, if the generated text is end in mid-sentence, the user can refresh the answer and the app to send the generated incomplete sentence to the API in order to continue generating the sentence completely.